## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [1]:
! uv pip install agentics-py
! uv pip install 'crewai[tools]'

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive

    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.12 environment at: /Users/nathansun/opt/anaconda3/envs/agentics
Audited 1 package in 141ms
Using Python 3.12.12 environment at: /Users/nathansun/opt/anaconda3/envs/agentics
Audited 1 package in 17ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [2]:
from ddgs import DDGS
from crewai.tools import tool


## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("web_earch")
def web_search(query: str) -> str:
    """Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))

## Define your agent, task and crew

Note that the agent is provided with memory = True This provides the agent with conversation history. By activating memory=True in the crew, we make sure that the agents persists across multiple crew session calls, with different input states. 

In [5]:
from crewai import Agent, Task, Crew, Process
from agentics import AG

# Create a conversational agent with a friendly role/goal
# chat_agent = Agent(
#     role="Helpful Assistant",
#     goal="Have a natural multi-turn conversation",
#     backstory="You are a friendly assistant that remembers context and asks for clarification when needed.",
#     llm=AG.get_llm_provider(),
#     memory=True,  # enable memory for conversational context.
# )

# # Define a simple Task that represents a single AI response
# task = Task(
#     description="Respond appropriately to user's message, maintaining context. {input}",
#     expected_output="A conversational reply",
#     agent=chat_agent,
#     tools=[web_search],
# )

chat_agent = Agent(
    role="Fundamental Analyst",
    goal="Use provided firm data to accurately predict future stock returns.",
    backstory="""
    As a fundamental financial equity
analyst your primary responsibility is to analyze the most
recent report of firm characteristics for all firms in the S&P 500. Some example features are firm size, book-to-market, and momentum.
Keep checking if you
have answered the users’ question to avoid looping.""",
    llm=AG.get_llm_provider(),
    memory=True,  # enable memory for conversational context.
)

# Define a simple Task that represents a single AI response
task = Task(
    description="Respond appropriately to user's message, maintaining context. {input}",
    expected_output="A detailed explanation behind the reasoning of the choices",
    agent=chat_agent,
    tools=[web_search],
)

crew = Crew(
    agents=[chat_agent],
    tasks=[task],
    memory=False,  # ensure conversation state persists across turns
)

## Run the crew in a conversation loop
Since memory has been set to True, the crew will get access to the previous round of conversations, exibiting a conversational behaviour

In [6]:
conversation = ""
while user_input := input("USER: ").strip():
    print("USER:", user_input)
    result = crew.kickoff(inputs={"input": conversation + user_input})
    print("AI:", result)
    conversation += f"User>{user_input}\nAI>{result}\n"

USER:  the book to market of a firm is increasing over time; what can you say about its stock price?


USER: the book to market of a firm is increasing over time; what can you say about its stock price?
AI: When the book-to-market ratio of a firm is increasing over time, it most commonly implies that its **stock price has been decreasing**.

Here is a detailed explanation of the reasoning behind this conclusion from a fundamental analyst's perspective:

### 1. Understanding the Book-to-Market Ratio

The book-to-market (B/M) ratio is a key financial metric used to find undervalued companies. It is calculated as follows:

**Book-to-Market Ratio = Book Value of Equity / Market Value of Equity**

*   **Book Value of Equity:** This is the company's net asset value as reported on its balance sheet (Total Assets - Total Liabilities). It is an accounting measure that reflects the historical cost of assets and accumulated profits. Book value typically changes slowly, quarter by quarter, as a company retains earnings or issues/repurchases shares.
*   **Market Value of Equity (Market Capitalizatio

KeyboardInterrupt: Interrupted by user